# CIFAR-10 Classification

Comparison of:
- CNN Model from Scratch
- ResNet50 with Transfer Learning

## Part 1: CNN from Scratch

In [1]:

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load and preprocess CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# Data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

# Manual validation split
x_train_part, x_val_part, y_train_part, y_val_part = train_test_split(
    x_train, y_train_cat, test_size=0.1, random_state=42
)
train_gen = datagen.flow(x_train_part, y_train_part, batch_size=64)
val_gen = datagen.flow(x_val_part, y_val_part, batch_size=64)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step


In [2]:

def build_scratch_model():
    model = models.Sequential([
        tf.keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.25),

        layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.25),

        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    return model


In [3]:

scratch_model = build_scratch_model()
scratch_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_scratch = scratch_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen)
)
test_loss_scratch, test_acc_scratch = scratch_model.evaluate(x_test, y_test_cat)
print(f"Improved Scratch CNN Test Accuracy: {test_acc_scratch * 100:.2f}%")


Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


704/704 ━━━━━━━━━━━━━━━━━━━━ 47s 53ms/step - accuracy: 0.3518 - loss: 2.1365 - val_accuracy: 0.5002 - val_loss: 1.3960
Epoch 2/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 29s 42ms/step - accuracy: 0.5424 - loss: 1.2880 - val_accuracy: 0.5920 - val_loss: 1.1382
Epoch 3/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 28s 40ms/step - accuracy: 0.6045 - loss: 1.1111 - val_accuracy: 0.6346 - val_loss: 1.0289
Epoch 4/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 34s 48ms/step - accuracy: 0.6395 - loss: 1.0200 - val_accuracy: 0.6776 - val_loss: 0.8933
Epoch 5/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 34s 39ms/step - accuracy: 0.6593 - loss: 0.9628 - val_accuracy: 0.6570 - val_loss: 0.9572
Epoch 6/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 28s 39ms/step - accuracy: 0.6790 - loss: 0.9107 - val_accuracy: 0.7062 - val_loss: 0.8328
Epoch 7/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 42s 41ms/step - accuracy: 0.6952 - loss: 0.8595 - val_accuracy: 0.7146 - val_loss: 0.7957
Epoch 8/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 40s 39ms/step - accuracy: 0.7114 - loss: 0.8306 - val_accurac

## Part 2: Fine-Tuned ResNet50

In [5]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

# Resize CIFAR-10 images for ResNet50 input
x_train_resized = tf.image.resize(x_train, (48, 48)).numpy()
x_test_resized = tf.image.resize(x_test, (48, 48)).numpy()

# Manual split for ResNet model
x_train_part_r, x_val_part_r, y_train_part_r, y_val_part_r = train_test_split(
    x_train_resized, y_train_cat, test_size=0.1, random_state=42
)

# Data augmentation for ResNet
datagen_resnet = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen_resnet.fit(x_train_resized)
train_gen_r = datagen_resnet.flow(x_train_part_r, y_train_part_r, batch_size=64)
val_gen_r = datagen_resnet.flow(x_val_part_r, y_val_part_r, batch_size=64)


In [6]:

resnet_base = ResNet50(include_top=False, weights='imagenet', input_shape=(48, 48, 3))
resnet_base.trainable = True

x = resnet_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(10, activation='softmax')(x)

resnet_model = Model(inputs=resnet_base.input, outputs=output)
resnet_model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [7]:

history_resnet = resnet_model.fit(
    train_gen_r,
    validation_data=val_gen_r,
    epochs=15,
    steps_per_epoch=len(train_gen_r),
    validation_steps=len(val_gen_r)
)
test_loss_resnet, test_acc_resnet = resnet_model.evaluate(x_test_resized, y_test_cat)
print(f"Improved ResNet50 Test Accuracy: {test_acc_resnet * 100:.2f}%")


Epoch 1/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 147s 122ms/step - accuracy: 0.4477 - loss: 1.7912 - val_accuracy: 0.1968 - val_loss: 2.4489
Epoch 2/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 90s 83ms/step - accuracy: 0.7683 - loss: 0.6917 - val_accuracy: 0.8220 - val_loss: 0.5179
Epoch 3/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 82s 84ms/step - accuracy: 0.8237 - loss: 0.5310 - val_accuracy: 0.8306 - val_loss: 0.4876
Epoch 4/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 61s 86ms/step - accuracy: 0.8495 - loss: 0.4491 - val_accuracy: 0.8418 - val_loss: 0.4836
Epoch 5/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 60s 85ms/step - accuracy: 0.8720 - loss: 0.3814 - val_accuracy: 0.8538 - val_loss: 0.4255
Epoch 6/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 60s 85ms/step - accuracy: 0.8848 - loss: 0.3444 - val_accuracy: 0.8464 - val_loss: 0.4789
Epoch 7/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 60s 84ms/step - accuracy: 0.8937 - loss: 0.3122 - val_accuracy: 0.8542 - val_loss: 0.4468
Epoch 8/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 80s 82ms/step - accuracy: 0.9039 - loss: 0.2857 

##  Final Comparison: CNN from Scratch vs. Fine-Tuned ResNet50

| Approach                  | Architecture               | Test Accuracy | Training Time | Key Observations                                   |
|---------------------------|----------------------------|---------------|----------------|----------------------------------------------------|
| CNN from Scratch          | 3 Conv + 2 Dense layers    | ~75–80%       | Low            | Basic feature extraction, limited generalization   |
| Fine-Tuned ResNet50       | Pretrained ResNet50 + Custom Head | ~88–92%       | Moderate        | Rich features from ImageNet, much better accuracy   |

 **Conclusion**: Fine-tuning ResNet50 dramatically boosts test accuracy on CIFAR-10 by transferring deep, hierarchical features learned from large-scale data. While it requires more computation, the performance gain is substantial compared to a basic CNN built from scratch.